# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv ('event_datafile_new.csv') that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Apache Cassandra code in the cells below

#### Create Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS cassandra 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace 

In [7]:
try:
    session.set_keyspace('cassandra')
except Exception as e:
    print(e)

## Below create tables to run the Apache Cassandra queries on the database.

### Query 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4



In [8]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = "CREATE TABLE IF NOT EXISTS music_library1 "
query1 = query1 + "(artist_name text, song_name text, song_length float, sessionid int, iteminsession int, PRIMARY KEY (sessionid, iteminsession))"
session.execute(query1)

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query1 = "INSERT INTO music_library1 (artist_name,song_name,song_length,sessionid,iteminsession)"
        query1 = query1 + " VALUES (%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query1, (line[0],line[9],float(line[5]),int(line[8]),int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query1 = "SELECT artist_name, song_name, song_length, sessionid FROM music_library1 WHERE sessionid=338 AND iteminsession=4"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_name, row.song_length, row.sessionid)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875 338


### Query 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = "CREATE TABLE IF NOT EXISTS music_library2 "
query2 = query2 + "(artist_name text, song_name text, userid int, iteminsession int, sessionid int, user_first text, user_last text, PRIMARY KEY ((userid),sessionid,iteminsession))"
session.execute(query2)                    

In [12]:
file2 = 'event_datafile_new.csv'

with open(file2, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    #print(sum(1 for line in f))
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query2 = "INSERT INTO music_library2 (artist_name, song_name,userid, iteminsession, sessionid, user_first, user_last)"
        query2 = query2 + " VALUES (%s,%s,%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query2, (line[0],line[9],int(line[10]),int(line[3]),int(line[8]),line[1],line[4]))
        
    

In [13]:
query2 = "SELECT * FROM music_library2 WHERE userid=10 AND sessionid=182;"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.userid, row.song_name,row.sessionid)

10 Keep On Keepin' On 182
10 Greece 2000 182
10 Kilometer 182
10 Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 182


### Query 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'    

In [14]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = "CREATE TABLE IF NOT EXISTS music_library3 "
query3 = query3 + "(song_name text, userid int, user_first text, user_last text, PRIMARY KEY (song_name, userid))"
session.execute(query3)       

In [15]:
file3 = 'event_datafile_new.csv'

with open(file3, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    #print(sum(1 for line in f))
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query3 = "INSERT INTO music_library3 (song_name, userid, user_first, user_last)"
        query3 = query3 + " VALUES (%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query3, (line[9],int(line[10]),line[1],line[4]))

In [16]:
query3 = "SELECT userid, user_first, user_last FROM music_library3 WHERE song_name='All Hands Against His Own';"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_first, row.user_last)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
#Drop music_library1
query4 = "DROP TABLE music_library1"
try:
    rows = session.execute(query4)
except Exception as e:
    print(e)

NameError: name 'psycopg2' is not defined

In [ ]:
#Drop music_library2
query5 = "DROP TABLE music_library2"
try:
    rows = session.execute(query5)
except Exception as e:
    print(e)

In [ ]:
#Drop music_library3
query6 = "DROP TABLE music_library3"
try:
    rows = session.execute(query6)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()